In [1]:
import time
import requests
import pandas as pd

# Code taken from http://compomics.github.io/projects/ms2pip_c/wiki/MS2PIP-Server-API
def run_ms2pip_server(peptides, frag_method, ptm_list, url='https://iomics.ugent.be/ms2pip/api/v2'):
    # Check if all columns are present in dataframe
    for col in ['spec_id', 'peptide', 'charge', 'modifications']:
        if col not in peptides.columns:
            print("{} is missing from peptides DataFrame".format(col))
            return None

    # Split-up into batches of 100 000 peptides (maximum MS2PIP Server accepts per request)
    batch_size = 100000
    result = pd.DataFrame()
    for i in list(range(0, len(peptides), batch_size)):
        print("Working on batch {}/{}".format(int(i / batch_size + 1), len(peptides) // batch_size + 1))
        peptides_batch = peptides.iloc[i:i+batch_size, :]

        # Combine data into dictionary for json post request
        input_data = {
            "params": {
                "frag_method": frag_method,
                "ptm": ptm_list
            },
            "peptides": peptides_batch.to_dict(orient='list')
        }

        # Post data to server and get task id
        response = requests.post('{}/task'.format(url), json=input_data)
        if 'task_id' not in response.json():
            if 'error' in response.json():
                print("Server error: {}".format(response.json()['error']))
            else:
                print("Something went wrong")
            return None
        task_id = response.json()['task_id']
        print("Received task id: {}".format(task_id))

        # Check server task status and get result when ready
        time.sleep(1)
        response = requests.get('{}/task/{}/status'.format(url, task_id))
        state = response.json()['state']

        if state != 'SUCCESS':
            print("Check MS2PIP Server status every 5 seconds", end='')
            state = 'PENDING'
            pending_count = 0

            while state == 'PENDING' or state == 'PROGRESS':
                time.sleep(5)
                print('.', end='')
                response = requests.get('{}/task/{}/status'.format(url, task_id))
                state = response.json()['state']

                # Do not keep looping if task state is stuck on PENDING, it might have failed silently
                if state == 'PENDING':
                    pending_count += 1
                if pending_count > 24:
                    print("\nSomething went wrong")
                    return None
            print('')

        if state == 'SUCCESS':
            response = requests.get('{}/task/{}/result'.format(url, task_id))
            result_batch = pd.DataFrame.from_dict(response.json())['ms2pip_out']
            result_batch = result_batch[['spec_id', 'charge', 'ion', 'ionnumber', 'mz', 'prediction']]
            result = result.append(result_batch)
            print("Result received", end='\n\n')
        else:
            error_message = response.json()['status']
            print("Something went wrong: {}".format(error_message))
            return None

    print("Finished with all batches!")

    return result

In [ ]:
PSMs = [
    [ 'HPTIQGFASVAQEETEILTADMDAER', '20150618_QEp1_LC7_PhGe_SA_Plate1B_11_7_mzXML_scan_18960', '-', 3],
    [ 'SSMSPTTNVLLSPLSVATALSALSLGAEQR', '20150824_QEp1_LC7_PhGe_SA_Plate4D_09_5_mzXML_scan_23618', '-', 3 ],
    [ 'NALTGLPSGLFQASATLDTLVLK', '20150618_QEp1_LC7_PhGe_SA_Plate1B_02_3_mzXML_scan_21535', '-', 2 ],
    [ 'MSLSSFSVNRPFLFFIFEDTTGLPLFVGSVK', '20150624_QEp1_LC7_PhGe_SA_Plate2A_06_2_mzXML_scan_18682', '-', 3 ],
    [ 'WLNVQLSPR' , '20150907_QEp1_LC7_PhGe_SA_Plate2D_38_7_mzXML_scan_16943', '-', 2 ],
    [ 'NTVLATWQPYTTSK', '20230601_ASC2_NEO3_X866_P244_R771_180min_uPAC110cm_27ms_DDA_S3_mzML_scan_96472', '-', 2 ],
    [ 'NTVLATWQPYSTSK', '20230601_ASC2_NEO3_X866_P244_R771_180min_uPAC110cm_27ms_DDA_S3_mzML_scan_92494', '-', 2 ]
]

psms_df = pd.DataFrame(data=PSMs, columns=['peptide', 'spec_id', 'modifications', 'charge'])

frag_method = 'HCDch2'
ptm_list = [
    'Oxidation,15.994915,M',
    'Carbamidomethyl,57.021464,C',
    'PhosphoS,79.966331,S',
    'PhosphoT,79.966331,T',
    'PhosphoY,79.966331,Y',
    'Glu->pyro-Glu,-18.010565,E',
    'Gln->pyro-Glu,-17.026549,Q',
    'Pyro-carbamidomethyl,39.994915,C',
    'Deamidated,0.984016,N',
    'iTRAQ,144.102063,N-term'
]

preds = run_ms2pip_server(psms_df, frag_method, ptm_list)

In [27]:
pred_obs_data = []

for i,spec_id in enumerate(preds['spec_id'][0]):
    pred_obs_data.append([spec_id, -1, 0, preds['mz'][0][i], preds['prediction'][0][i], preds['ion'][0][i][0].lower() + str(preds['ionnumber'][0][i]) + ('++' if preds['ion'][0][i].endswith('2') else '')])

In [22]:
import numpy as np

peak_threshold_intensity = 0.02
peak_threshold_intensity_aligned = 0.01

def scale_intensities(spectrum):
	intensities = spectrum[1]
	max_ints = max(intensities)
	
	scaled_intensities = []
	for ints in  intensities:
		scaled_intensities.append(ints / max_ints)
	
	scaled_spectrum = np.array([spectrum[0].copy(), scaled_intensities, spectrum[2].copy()])
	
	return scaled_spectrum

def find_aligned_peaks(m_peaks, p_peaks, ppm_error_threshold, min_intensity):
	aligned_m_mzs = []
	aligned_m_intensities = []
	aligned_p_mzs = []
	aligned_p_intensities = []
	aligned_m_indices = []
	aligned_p_indices = []

	# do not match peaks with intensity below a threshold
	all_intensities = m_peaks[1,:].tolist()
	m_peaks_mask = [ intensity >= min_intensity for intensity in all_intensities ]
	m_peaks_filtered = m_peaks[0,:][m_peaks_mask]

	for i in range(p_peaks.shape[1]):
		peak = [p_peaks[0,i], p_peaks[1,i]]
		if peak[1] == 0:
			continue
		ppm_errors = np.abs(m_peaks_filtered - peak[0])
		ppm_errors = np.array((1000000.0/peak[0]) * ppm_errors)
		ppm_errors = ppm_errors.tolist()
		min_error = min(ppm_errors)
		if min_error <= ppm_error_threshold:
			# find the corresponding closest peak in the original array
			closest_mass = m_peaks_filtered[ppm_errors.index(min_error)]
			closest_mass_index = m_peaks[0,:].tolist().index(closest_mass)

			aligned_m_mzs.append(closest_mass)
			aligned_m_intensities.append(m_peaks[1,closest_mass_index])
			aligned_p_mzs.append(peak[0])
			aligned_p_intensities.append(peak[1])
			aligned_m_indices.append(closest_mass_index)
			aligned_p_indices.append(i)

	return np.array([aligned_m_mzs, aligned_m_intensities, aligned_p_mzs, aligned_p_intensities]), aligned_m_indices, aligned_p_indices 
	
def normalize_peaks(m_peaks, p_peaks, aligned_peaks):
	measured_median = 0.0
	predicted_median = 0.0

	if (aligned_peaks.shape[1] == 0):
		measured_median = np.median(m_peaks[1,:])
		predicted_median = np.median(p_peaks[1,:])
	else:
		measured_median = np.median(aligned_peaks[1,:])
		predicted_median = np.median(aligned_peaks[3,:])

	scaled_m_peaks = np.copy(m_peaks)
	scaled_p_peaks = np.copy(p_peaks)

	scaled_m_peaks[1,:] = scaled_m_peaks[1,:] / measured_median
	scaled_p_peaks[1,:] = scaled_p_peaks[1,:] / predicted_median

	return scaled_m_peaks, scaled_p_peaks

In [32]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

plt.rc('axes', titlesize=35) #fontsize of the title
plt.rc('axes', labelsize=35) #fontsize of the x and y labels
plt.rc('font', size=25)

def plot_spectrum(m_peaks, p_peaks, aligned_m_indices, aligned_p_indices, plot_filename, title, observed_ions, predicted_ions):
	plt.figure(figsize=(20,15))
	plt.tight_layout()
	#print(observed_ions)

	all_matched_m_indices = [ i for i in range(m_peaks.shape[1]) if (observed_ions[i] != '') or (i in aligned_m_indices)]
	not_aligned_m_indices = [ i for i in range(m_peaks.shape[1]) if (i not in all_matched_m_indices)]
	not_aligned_p_indices = [ i for i in range(p_peaks.shape[1]) if (i not in aligned_p_indices)]

	# plot everything that is not aligned first
	for i in not_aligned_m_indices:
		if (m_peaks[1,i] > peak_threshold_intensity): #and (m_peaks[0,i] < 2300):
			plt.plot([m_peaks[0,i], m_peaks[0,i]], [0.0, m_peaks[1,i]], c=mcolors.CSS4_COLORS['darkgray'], linewidth = 1)

	# mark not aligned predicted peaks in red
	for i in not_aligned_p_indices:
		if (p_peaks[1,i] > peak_threshold_intensity_aligned): #and (p_peaks[0,i] < 2300):
			plt.plot([p_peaks[0,i], p_peaks[0,i]], [0.0, -p_peaks[1,i]], c=mcolors.CSS4_COLORS['crimson'], linewidth = 4)
			if (predicted_ions[i] != ''):
				plt.text(p_peaks[0,i] - 0.01, - p_peaks[1,i] - 0.05, predicted_ions[i])

	# plot the aligned peaks on top
	for i in all_matched_m_indices:
		#if (m_peaks[1,i] > peak_threshold_intensity_aligned):
			if i in aligned_m_indices:
				plt.plot([m_peaks[0,i], m_peaks[0,i]], [0.0, m_peaks[1,i]], c=mcolors.CSS4_COLORS['darkblue'], linewidth = 4)
				#if (m_peaks[1,i] > 0.05):
				#	plt.text(m_peaks[0,i] - 0.01, m_peaks[1,i] + 0.02, observed_ions[i])
			elif (m_peaks[1,i] > peak_threshold_intensity_aligned):
				plt.plot([m_peaks[0,i], m_peaks[0,i]], [0.0, m_peaks[1,i]], c=mcolors.CSS4_COLORS['orchid'], linewidth = 4)
				#if (m_peaks[1,i] > 0.05):
				#	plt.text(m_peaks[0,i] - 0.01, m_peaks[1,i] + 0.02, observed_ions[i])

	for i in aligned_p_indices:
		#if (p_peaks[1,i] > peak_threshold_intensity_aligned):
			plt.plot([p_peaks[0,i], p_peaks[0,i]], [0.0, -p_peaks[1,i]], c=mcolors.CSS4_COLORS['mediumseagreen'], linewidth = 4)
			if (predicted_ions[i] != ''): # and (((p_peaks[1,i] > 0.01) and (p_peaks[0,i] < 1300)) or ((p_peaks[1,i] > 0.08) and (p_peaks[0,i] >= 1300) and (p_peaks[0,i] < 1900)) or ((p_peaks[1,i] > 0.01) and (p_peaks[0,i] > 1900))):
				plt.text(p_peaks[0,i] - 0.01, -p_peaks[1,i] - 0.05, predicted_ions[i])
				
	plt.axhline(0, color='black')
	plt.xlabel('m/z')
	plt.ylabel('Scaled intensity')
	#plt.title(title)
	plt.savefig(plot_filename, dpi=600, transparent=False)
	plt.close()

In [28]:
import json

for specID in psms_df['spec_id'].tolist():
    infile = open('../data/spectra/' + specID + '.json', 'r')
    peaks_parsed = json.load(infile)
    infile.close()

    for peak in peaks_parsed['peaks']:
        pred_obs_data.append([specID, 1, 0, peak[0], peak[1], ''])

In [33]:
peaks_df = pd.DataFrame(data=pred_obs_data, columns=['PSMId','measuredLabel','matchedLabel','mz','intensity','ion'])

observed_spectra = {}
predicted_spectra = {}
observed_ions = {}
predicted_ions = {}

plots_folder = '../data/spectra'

for psm in psms_df['spec_id'].tolist():
	obs_spectrum_df = peaks_df[(peaks_df['PSMId'] == psm) & (peaks_df['measuredLabel'] == 1)]
	obs_mz = obs_spectrum_df['mz'].tolist()
	obs_int = obs_spectrum_df['intensity'].tolist()
	obs_ion = obs_spectrum_df['ion'].tolist()
	obs_matched = obs_spectrum_df['matchedLabel'].tolist()
	
	observed_spectra[psm] = np.array([obs_mz, obs_int, obs_matched])
	
	pred_spectrum_df = peaks_df[(peaks_df['PSMId'] == psm) & (peaks_df['measuredLabel'] == -1)]
	pred_mz = pred_spectrum_df['mz'].tolist()
	pred_int = pred_spectrum_df['intensity'].tolist()
	pred_ion = pred_spectrum_df['ion'].tolist()
	pred_matched = pred_spectrum_df['matchedLabel'].tolist()
	
	predicted_spectra[psm] = np.array([pred_mz, pred_int, pred_matched])
	
	observed_ions[psm] = obs_ion
	predicted_ions[psm] = pred_ion
	
for psm in psms_df['spec_id'].tolist():
	measured_peaks = scale_intensities(observed_spectra[psm])
	predicted_peaks = scale_intensities(predicted_spectra[psm])
	obs_ions = observed_ions[psm]
	pred_ions = predicted_ions[psm]

	ppm_error_threshold = 20.0
	aligned_peaks, aligned_m_indices, aligned_p_indices = find_aligned_peaks(measured_peaks, predicted_peaks, ppm_error_threshold, 0.01)
	
	#scaled_m_peaks, scaled_p_peaks = normalize_peaks(measured_peaks, predicted_peaks, aligned_peaks)
	scaled_m_peaks = measured_peaks
	scaled_p_peaks = predicted_peaks
	
	plot_filename = plots_folder + '/' + psm + '_spectrum_alignment.png'
	title = ''
	
	plot_spectrum(scaled_m_peaks, scaled_p_peaks, aligned_m_indices, aligned_p_indices, plot_filename, title, obs_ions, pred_ions)